In [6]:
import pandas as pd
import os
from scapy.all import rdpcap, IP, TCP
import json
from tqdm.auto import tqdm

In [7]:
# TODO: 출력을 참고하시오.

TEST_PCAP_PATH = "./CSTNET_PCAP/51.la/51.la_1.pcap"
packets = rdpcap(TEST_PCAP_PATH)
# packets[0][1].show()
pkt_tool = packets[0]

pkt_tool[IP].src

'59.109.99.212'

In [8]:
def normalize_key(src_ip, src_port, dst_ip, dst_port, proto):
	"""
	세션 키 정규화:
	  - endA, endB 두 단말(endpoint)을 정의한다.
	  - 목표: '동적(임시, ephemeral) 포트를 사용하는 쪽'을 endA로 정규화한다.
	    → 일반적으로 클라이언트(동적 포트)가 endA, 서버(고정 포트)가 endB가 되도록 설정한다.
	  - 두 포트 모두 동적 포트가 아닐 경우, 포트 번호가 더 큰 쪽을 endA로 설정한다.
	"""
	src_port = int(src_port)
	dst_port = int(dst_port)
	a = (src_ip, src_port)  # 현재 패킷의 (src_ip, src_port)
	b = (dst_ip, dst_port)  # 현재 패킷의 (dst_ip, dst_port)
	endA = (-1, -1)
	endB = (-1, -1)

	# TODO: '동적 포트를 사용하는 쪽' 혹은 '포트 번호가 더 큰 쪽'을 endA로 설정하도록 조건을 작성하시오.
	# 조건이 True이면 (endA, endB) = (a, b), 그렇지 않으면 (b, a)로 설정하시오.

	# 출발포트가 동적 포트일때
	if src_port >= 1024 and dst_port < 1024:
		endA, endB = a, b
	# 출발 포트가 고정 포트일때
	elif src_port < 1024 and dst_port >= 1024:
		endA, endB = b, a
	# 둘 다 동적 혹은 둘 다 고정
	else:
		if src_port > dst_port:
			endA, endB = a, b
		else:
			endA, endB = b, a

	return (endA[0], endA[1], endB[0], endB[1], proto)


def get_pkt_len(pkt):
	"""
	패킷 길이 계산:
	  - 가능한 경우, IP 헤더의 total length(IP.len)를 사용한다.
	  - 해당 필드가 없으면 파싱된 패킷의 실제 바이트 길이(len(pkt))를 사용한다.
	"""
	try:
		if IP in pkt and hasattr(pkt[IP], "len") and pkt[IP].len is not None:
			return int(pkt[IP].len)
	except Exception:
		pass
	return int(len(pkt))


def to_df(path):
	"""
	PCAP 파일을 읽어 세션 단위로 데이터를 정리하고, DataFrame 형태로 변환한다.
	세션 구조:
	{
	  "seq": [{"t": ts, "len": L, "dir": +1/-1}, ...],
	  "endA": (ip, port),   # 동적 포트를 사용하는 쪽 (클라이언트)
	  "endB": (ip, port),   # 서비스 포트를 사용하는 쪽 (서버)
	  "proto": "TCP"
	}
	"""
	packets = rdpcap(path)   # PCAP 파일 읽기
	sessions = {}            # 세션 정보를 저장할 딕셔너리

	for pkt in packets:
		if IP not in pkt:
			continue

		# TODO: PCAP 파일 구조를 확인한 뒤, source/destination의 IP와 Port 번호를 추출하시오.
		# (2번째 셀의 출력 예시를 참고하여 올바른 필드를 채워 넣으시오.)
		src_ip = pkt_tool[IP].src
		dst_ip = pkt_tool[IP].dst
		proto  = None
		src_port = None
		dst_port = None

		if TCP in pkt:
			proto = "TCP"
			src_port = pkt_tool[IP].sport
			dst_port = pkt_tool[IP].dport
		else:
			# 이번 과제에서는 TCP만 처리한다.
			continue

		key = normalize_key(src_ip, src_port, dst_ip, dst_port, proto)

		endA = (key[0], key[1])
		endB = (key[2], key[3])

		# TODO: 현재 패킷의 (src_ip, src_port)가 endA와 같으면 +1, 그렇지 않으면 -1이 되도록 작성하시오.
		direction = 1 if(src_ip, src_port) == endA else -1
		
		pkt_len = get_pkt_len(pkt) # 패킷 길이
		ts = float(pkt.time)  # 패킷 타임스탬프

		# 세션이 존재하지 않으면 새로 생성
		if key not in sessions:
			sessions[key] = {
				"seq": [],
				"endA": endA,
				"endB": endB,
				"proto": proto
			}

		sessions[key]["seq"].append({"t": ts, "len": pkt_len, "dir": direction})

	# 세션별로 패킷을 시간 순으로 정렬하고, 시퀀스 데이터를 추출
	records = []
	for data in sessions.values():
		# TODO: 세션 내의 패킷을 시간 순으로 정렬하시오.
		seq = sorted(data['seq'], key=lambda x: x["t"])

		# TODO: 정렬된 데이터를 이용해 시간, 길이, 방향 시퀀스를 각각 추출하시오.
		ts_seq   = [x["t"] for x in seq]  # 시간 시퀀스
		len_seq  = [x["len"] for x in seq]  # 길이 시퀀스
		dir_seq  = [x["dir"] for x in seq]  # 방향 시퀀스

		# 리스트 형태의 데이터를 JSON 문자열로 변환 (CSV 저장 시 안전하게 처리하기 위함)
		records.append({
			"src_ip": data["endA"][0],
			"src_port": data["endA"][1],
			"dst_ip": data["endB"][0],
			"dst_port": data["endB"][1],
			"protocol": data["proto"],
			"num_packets": len(seq),
			"time_sequence": json.dumps(ts_seq),
			"length_sequence": json.dumps(len_seq),
			"direction_sequence": json.dumps(dir_seq),
		})

	df = pd.DataFrame(records)
	return df


In [9]:
PCAP_PATH = "./CSTNET_PCAP/"
LABEL_PATH = os.listdir(PCAP_PATH)

final_df = pd.DataFrame()

for LABEL in tqdm(LABEL_PATH, desc=f'Data preprocessing'):
	pcap_files_path = os.path.join(PCAP_PATH, LABEL)
	pcap_files = os.listdir(pcap_files_path)
	
	# 파일의 개수가 400기 미만이면 전처리 진행하지 마시오.
	if len(pcap_files) < 400:
		continue
	
	flow_data = pd.DataFrame()

	for file in pcap_files:
		path = os.path.join(pcap_files_path, file)
		if file.split('.')[-1] != "pcap":
			continue
		# Label의 이름을 추출한다.
		tmp = file.split('.')
		app_name = ''
		for i in tmp[:-1]:
			app_name += i + '.'
		app_name = app_name[: -1]
		app_name = app_name.split('_')[0]
		
		# 전처리를 진행한다.
		df = to_df(path)

		# 각 데이터의 어떤 데이터로 왔는지와 라벨을 저장한다.
		df['from'] = file
		df['app_name'] = app_name

		# 데이터를 결합한다.
		flow_data = pd.concat([flow_data, df], ignore_index=True)

	# 라벨 별 데이터를 결합한다.
	final_df = pd.concat([final_df, flow_data], ignore_index=True)


Data preprocessing: 100%|██████████| 120/120 [19:18<00:00,  9.65s/it] 


In [10]:
# CSV로 파일 저장한다.

final_df.to_csv('./CSTNET_df.csv', index=False)